# Testing out OPT on Self-Ask Decomposition

Lets see how large of a model we can effectively use!

In [1]:
from transformers import AutoTokenizer
from llm_serving.model.wrapper import get_model

In [ ]:
#size="13b"
# Load the tokenizer. All OPT models with different sizes share the same tokenizer
tokenizer = AutoTokenizer.from_pretrained(f"facebook/opt-66b")
tokenizer.add_bos_token = False

# Load the model. Alpa automatically downloads the weights to the specificed path

model = get_model(model_name=f"alpa/opt-66b", path="/scratch/ddr8143/opt_weights/")

Downloading:   0%|          | 0.00/685 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/676 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/221 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/9.85G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.85G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/6.36G [00:00<?, ?B/s]

In [ ]:
# Generate
prompt = """Question: Harmony Korine was both screenwriter and director of what movie?
Can this be decomposed: Yes.
Question 1: Harmony Korine was the screenwriter of what movie?
Question 2: Harmony Korine was the director of what movie?
So the final answers are: Spring Breakers, Trash Humpers, Julien Donkey-Boy, Mister Lonely, The Beach Bum, Gummo

Question: Who was both a graduate from Ananda College and University of Ceylon?
Can this be decomposed: Yes.
Question 1: Who graduated from Ananda College?
Question 2: Who graduated from University of Ceylon?
So the final answers are: Tissa Vitharana, V. K. Samaranayake, Chandre Dharma-wardana, D. Shelton A. Gunaratne, K. D. Arulpragasam, J. B. Disanayake

Question: Which movie had K. S. L. Swamy as its director and Vijaya Bhaskar as its musical composer?
Can this be decomposed: Yes.
Question 1: Which movie has K. S. L. Swamy as its director?
Question 2: Which movie has Vijaya Bhaskar as its musical composer?
So the final answers are: Anna Thamma, Thulasi, Devaru Kotta Thangi, Manku Dinne, Lagna Pathrike, Malaya Marutha, Mugdha Manava, Banashankari, Aaru Mooru Ombhatthu, Bhagya Jyothi, Bhagyada Bagilu, Bhale Adrushtavo Adrushta, Maagiya Kanasu, Mithileya Seetheyaru, Arishina Kumkuma, Makkala Bhagya, Jimmy Gallu, Huli Hejje, CID 72, Lakshmi Saraswathi, Thoogudeepa

Question: What music was composed by Devi Sri Prasad and  produced by Dil Raju?
Can this be decomposed: Yes.
Question 1: What music was composed by Devi Sri Prasad?
Question 2: What music was produced by Dil Raju?
So the final answers are: Arya, Bommarillu, Nenu Local, Mr. Perfect, Bhadra, Yevadu, Maharshi, Middle Class Abbai

Question: What movie did Irwin Allen both direct and produce?
Can this be decomposed:"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
output = model.generate(
    input_ids=input_ids,
    max_length=600,
    do_sample=True,
)
generated_string = tokenizer.batch_decode(output, skip_special_tokens=True)

print(generated_string[0])